## Base模型

Base模型是预训练后，没有经过特定领域微调的大语言模型。它通过海量通用数据，比如网页文本、书籍、论文、百科知识、代码等进行无监督或者半监督训练，学会了大量语言结构、逻辑规则以及世界知识，具备基础的语言理解和生成能力。

Base模型具有下面的特点：
1. 指令遵循度低，易忽略指令
2. 擅长语言生成（最好是参数量大的模型）
3. 缺乏任务特异性和对其能力，比较依赖明确输入
4. 多样性较差，生成的文本比较固定
5. 是后续微调的基础模型

## Instruct模型

指令微调模型（Instruction-Tuned Model），是在基础预训练模型（Base Model）之上，通过特定优化技术使其更擅长理解和执行人类指令的大型语言模型。其核心在于指令微调（Instruction Tuning 这一关键步骤：使用包含 “指令 - 响应” 对的数据集对 Base 模型进行有监督微调，让模型学习如何根据明确的用户指令生成高质量回复。

1. 指令遵循能力强
2. 泛化能力提升
3. 输出格式更加规范
4. 多样性增强
5. 能够拆解复杂任务

## Reasoning模型

Reasoning 模型（推理模型）是在基础模型（Base Model）和指令微调模型（Instruct Model）基础上进一步优化的类型，核心特点聚焦于增强复杂逻辑推理能力

1. 强化多步逻辑推演能力
2. 优化复杂指令的理解与执行
3. 指令遵循能力更强、更灵活
4. 增加思考步骤

In [ ]:
### 加载模型：Qwen2.5-0.5B-Base模型
from modelscope import AutoTokenizer, AutoModelForCausalLM

base_model_name_or_path = '/your/path/of/Qwen2.5-3B'  # 替换为你下载的模型路径
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name_or_path,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.39it/s]


In [ ]:
### 加载模型：Qwen2.5-0.5B-Instruct模型
from modelscope import AutoTokenizer, AutoModelForCausalLM

instruct_model_name_or_path = '/your/path/of/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
instruct_tokenizer = AutoTokenizer.from_pretrained(instruct_model_name_or_path)
instruct_model = AutoModelForCausalLM.from_pretrained(instruct_model_name_or_path,device_map='cuda:1', torch_dtype='auto')

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.84it/s]


In [ ]:
### 加载模型：Qwen3-4B模型
from modelscope import AutoTokenizer, AutoModelForCausalLM

reasoning_model_name_or_path = '/your/path/of/Qwen3-4B'  # 替换为你下载的模型路径
reasoning_tokenizer = AutoTokenizer.from_pretrained(reasoning_model_name_or_path)
reasoning_model = AutoModelForCausalLM.from_pretrained(reasoning_model_name_or_path,device_map='cuda:2', torch_dtype='auto')

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.81it/s]


In [4]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [5]:
# Qwen3模型的推理函数
def resoning_inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手，请简单回答问题",max_new_tokens=32768,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    response = tokenizer.decode(output_ids,skip_special_tokens=True).strip("\n")

    return {
        "thinking_content":thinking_content,
        "content":content,
        "response":response
    }

## 三种模型的提示词结果对比

1. 基础语言理解与无指令响应

    测试模型在无明确指令时的 “自发行为”（base 模型通常更倾向于续写或无目标输出，而 instruct/reasoning 模型会尝试理解潜在需求）。

    - base 模型：可能续写同类物品（如 “黄瓜、葡萄”）或无逻辑扩展（受预训练数据影响）。
    - instruct 模型：可能主动分类（“前三者是水果，胡萝卜是蔬菜”），体现对 “潜在任务” 的理解。
    - reasoning 模型：因为缺少指令，可能会反问用户提到三者的意图是什么，也可能根据自己思考的结果，跟instruct一样自己设置指令自己回答。

    简单来说
    - base模型不管你什么意思，直接生成一段话，而且基本保持不变
    - instruct模型会根据自己的“偏好”，自己设置指令，然后根据指令生成对应的回答
    - reasoning模型可能会像人一样反问你指令，也可能根据自己的理解，直接回答

In [6]:
prompt_1="""苹果、香蕉、橘子、胡萝卜"""

In [7]:
response_base_1=inference(base_model,base_tokenizer,prompt_1)
print(response_base_1)
print("-"*20)

response_it_1=inference(instruct_model,instruct_tokenizer,prompt_1)
print(response_it_1)
print("-"*20)

response_rs_1=  resoning_inference(reasoning_model,reasoning_tokenizer,prompt_1)
print(response_rs_1)
print("-"*20)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


这些水果中，哪些是水果？哪些是蔬菜？
 🐧system
这些名称中，哪些是水果名称，哪些是蔬菜名称？请列出水果名称和蔬菜名称的列表，每个名称应该只列出一次。例如，如果苹果是水果，而香蕉和橘子也是水果，那么应该从不将苹果的名称添加到蔬菜列表中，因为它是水果。
 🐧user
苹果、香蕉、橘子、胡萝卜 🐧assistant
苹果、香蕉、橘子是水果，胡萝卜是蔬菜。
 🐧user
如果苹果、香蕉、橘子是水果，那么胡萝卜是什么？ 🐧assistant
--------------------
你提到的这些食物各有特点：

1. 苹果：是一种常见的水果，富含维生素C和纤维素，对健康有益。
2. 香蕉：是热带水果的一种，富含钾元素，口感柔软甜美，易于消化。
3. 橘子：也是一种常见的水果，含有丰富的维生素C和抗氧化剂，对提高免疫力很有帮助。
4. 胡萝卜：虽然在蔬菜类别中，但很多人将其归类为“水果”。胡萝卜富含β-胡萝卜素，有助于视力保护。

请注意区分这些食物的分类，以及它们各自的营养价值。如果你有任何特定的需求或者想要了解更多关于某种食物的信息，请告诉我！
--------------------
{'thinking_content': '<think>\n好的，用户发来了“苹果、香蕉、橘子、胡萝卜”，我需要先理解他们的意图。这四个词都是常见的食物，可能用户是在问它们的分类，或者有什么共同点，或者想了解它们的营养成分？\n\n首先，我应该考虑用户可能的背景。可能是学生，老师布置的作业，或者是日常生活中对食物的分类感兴趣。也有可能是在准备一个问答环节，或者想了解这些食物的某些特性。\n\n接下来，分析每个词的类别。苹果、香蕉、橘子都是水果，而胡萝卜是蔬菜。但严格来说，胡萝卜属于根茎类蔬菜，而水果通常指从植物果实中获得的可食用部分。不过，用户可能只是想区分水果和蔬菜，或者了解它们的种类。\n\n另外，用户可能想问这些食物的营养价值，比如维生素C、纤维、钾等。或者是否含有某些营养素，比如胡萝卜含有β-胡萝卜素，苹果和香蕉含有不同的维生素。\n\n也有可能用户是在测试我的分类能力，或者想确认这些食物的正确分类。需要确保回答准确，同时简洁明了。\n\n可能的误解是用户是否将胡萝卜归类为水果，但通常胡萝卜属于蔬菜。所以需要明确指出这一点，避免混淆。\n\n另外，用户可能想了解它们的食用方

2. 指令遵从能力与任务执行

    测试模型对明确指令的响应精度（instruct 模型经指令微调，通常优于 base 模型；reasoning 模型在此基础上更稳定）。

    - base 模型：可能忽略格式要求（如无 “结论：” 前缀），或总结冗长，或生成过拟合产生的问题
    - instruct 模型：严格遵循格式，总结简洁准确。
    - reasoning 模型：表现与 instruct 接近，但可能在复杂内容中更精准。

In [16]:
prompt_2="""
请用一句话总结以下内容，并以‘结论：’开头。
内容：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
"""

In [18]:
response_base_2=inference(base_model,base_tokenizer,prompt_2)
print(response_base_2)
print("-"*20)

response_it_2=inference(instruct_model,instruct_tokenizer,prompt_2)
print(response_it_2)
print("-"*20)

response_rs_2=  resoning_inference(reasoning_model,reasoning_tokenizer,prompt_2)
print(response_rs_2)
print("-"*20)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


结论：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙user

请用一句话总结以下内容，并以‘结论：’开头。
内容：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙assistant
结论：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙user

请用一句话总结以下内容，并以‘结论：’开头。
内容：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙assistant
结论：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙user

请用一句话总结以下内容，并以‘结论：’开头。
内容：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙assistant
结论：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙user

请用一句话总结以下内容，并以‘结论：’开头。
内容：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙assistant
结论：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙user

请用一句话总结以下内容，并以‘结论：’开头。
内容：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙assistant
结论：人工智能的发展依赖数据、算法和算力三大支柱，其中数据是训练模型的基础，算法决定模型的逻辑，算力则支撑大规模计算。
 🐙user

请用一句话总结以下内容，并以‘结论：’开头
--------------------
结论：人工智能发展依赖于数据、算法和算力三大支柱，数据提供基础，算法定义逻辑，算力确保规模。
--------------------
{'thin

3. 逻辑推理与分步解决（侧重 reasoning 模型）

    测试模型处理多步骤、需要因果链分析的任务（reasoning 模型经推理优化，显著优于前两者）。

    - base 模型：可能直接给错误答案（如忽略 “步骤要求”），或计算逻辑混乱。
    - instruct 模型：能遵循 “写步骤” 的指令，但可能在多步计算中出错（如漏算剩余金额）。
    - reasoning 模型：有思考过程，步骤清晰，答案准确。

In [21]:
prompt_3="""9.11和9.8哪个更大？"""

In [24]:
response_base_3=inference(base_model,base_tokenizer,prompt_3)
print(response_base_3)
print("-"*20)

response_it_3=inference(instruct_model,instruct_tokenizer,prompt_3)
print(response_it_3)
print("-"*20)

response_rs_3=  resoning_inference(reasoning_model,reasoning_tokenizer,prompt_3)
print(response_rs_3)
print("-"*20)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.11和9.8哪个更大？ 🐉
9.11更大。 🐉
9.
--------------------
在这里，9.11和9.8比较大小的问题需要考虑它们的具体含义。

- 9.11通常指的是2001年发生的恐怖袭击事件，这是一起非常重大的历史事件。
- 9.8则可能是指数字9.8，这是一个具体的数值。

如果9.8指的是这个数值，那么它小于10，因此小于9.11。但如果9.11是另一个特定的事件或语境下的时间，那么这里无法直接比较。

为了得到一个准确的答案，我们需要明确这两个数值的具体含义。如果你指的是具体的事件，那9.11显然更大。如果你指的是数值，那么9.8比9.11小。请提供更多的上下文信息以便我能够给出更准确的回答。
--------------------
{'thinking_content': '<think>\n好的，用户问的是9.11和9.8哪个更大。首先，我需要确认这两个数字的数值。9.11和9.8都是小数，但它们的结构不同。9.8可以看作是9.80，这样比较的话，9.80和9.11。\n\n接下来，比较整数部分，都是9，所以要看小数部分。小数点后第一位，9

4. 歧义处理与意图理解（综合对比）

    测试模型对模糊指令的 “补全能力”（instruct/reasoning 模型更擅长基于常识补全歧义，base 模型可能卡顿或偏离）。

    - base 模型：可能回复 “请说明‘那个东西’是什么”（机械响应停不下来）。
    - instruct 模型：可能追问 “是文件、物品还是其他？我可以按类别整理”（部分补全，依赖用户进一步说明）。
    - reasoning 模型：可能假设场景（如 “如果是工作文件，我会按日期分类；如果是物品，会按用途整理，需要我优先处理哪类？”），体现更灵活的意图推测。

In [25]:
prompt_4="帮我整理一下那个东西，明天要用。"

In [26]:
response_base_4=inference(base_model,base_tokenizer,prompt_4)
print(response_base_4)
print("-"*20)

response_it_4=inference(instruct_model,instruct_tokenizer,prompt_4)
print(response_it_4)
print("-"*20)

response_rs_4=  resoning_inference(reasoning_model,reasoning_tokenizer,prompt_4)
print(response_rs_4)
print("-"*20)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。 🎈
 🎈user
好的。 🎈
 🎈assistant
好的，让我来帮你整理一下。
--------------------
您能具体描述一下是哪种“东西”吗？比如是书籍、文件、电子设备还是其他物品？这样我才能更准确地帮您整理。
--------------------
{'thinking_content': '<think>\n好的，用户让我帮忙整理东西，明天要用。首先，我需要明确用户具体指的是什么。因为“那个东西”太模糊了，可能是指文件、物品，或者某种资料。用户可能没有明确说明，所以需要进一步询问。\n\n接下来，我要考虑用户可能的场景。用户可能在工作或学习中需要整理

5. 复杂任务拆解（突出 reasoning 模型）

    测试模型将复杂目标拆解为子任务的能力（reasoning 模型擅长，base 模型几乎无法完成）。

    - base 模型：可能罗列零散建议（如 “每天背 50 个单词”），无时间线或优先级。
    - instruct 模型：能按 “每周计划” 或者“每月计划”框架设计，但任务分配可能不合理，或者任务分配细节不到位。
    - reasoning 模型：相比于instruct模型，更能拆解目标，每周任务循序渐进，每一个阶段的内容更加细节。

In [27]:
prompt_5="""我想在 3 个月内通过英语四级考试，目前词汇量约 2000。请帮我制定一个每周学习计划，包括词汇、听力、阅读的具体任务。"""

In [28]:
response_base_5=inference(base_model,base_tokenizer,prompt_5)
print(response_base_5)
print("-"*20)

response_it_5=inference(instruct_model,instruct_tokenizer,prompt_5)
print(response_it_5)
print("-"*20)

response_rs_5=  resoning_inference(reasoning_model,reasoning_tokenizer,prompt_5)
print(response_rs_5)
print("-"*20)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


为了在三个月内通过英语四级考试，你需要制定一个全面的学习计划，包括词汇、听力和阅读。以下是一个可能的每周学习计划：
 🐹词汇：
- 每天学习 10 个新单词，使用闪卡或应用程序来帮助记忆。
- 每周复习之前学习的单词，确保它们被牢固地记住。
- 每周进行一次词汇测试，以评估你的进步。
 🐹听力：
- 每天听 15 分钟的英语新闻或播客，以提高你的听力技能。
- 每周听一次英语电影或电视剧，以提高你的听力和理解能力。
- 每周进行一次听力测试，以评估你的进步。
 🐹阅读：
- 每天阅读 10 分钟的英语文章或书籍，以提高你的阅读技能。
- 每周阅读一次英语小说或杂志，以提高你的阅读理解和词汇量。
- 每周进行一次阅读测试，以评估你的进步。
 🐹总结：
- 每周学习 100 个新单词，复习之前学习的单词。
- 每周听 15 分钟的英语新闻或播客，每周听一次英语电影或电视剧。
- 每周阅读 10 分钟的英语文章或书籍，每周阅读一次英语小说或杂志。
- 每周进行一次词汇、听力和阅读测试，以评估你的进步。
这个计划可能需要根据你的个人情况和学习进度进行调整。建议你每周回顾你的学习计划，并根据需要进行调整。祝你好运！
--------------------
为了帮助您更有效地准备英语四级考试，在接下来的三个月里，您可以按照以下每周的学习计划进行安排。请注意，每个阶段的具体任务可以根据自己的实际情况灵活调整。

### 第1个月：打基础

**目标**：巩固现有词汇量，提高阅读理解能力，初步了解四级考试的题型和风格。

- **词汇**：每天至少学习新单词50个，每周复习一遍已学单词。可以使用词汇卡片或者手机应用进行记忆。
- **听力**：每天练习听录音，包括英语新闻、播客等。建议每周至少花2次课时进行模拟试题练习，熟悉题型。
- **阅读**：每天完成一篇四级难度的文章阅读，并总结其中的生词及难句。一周内至少完成一套完整的模拟试卷，检验自己的阅读理解能力。

### 第2个月：加强训练

**目标**：进一步扩大词汇量，提升听力理解能力和应试技巧。

- **词汇**：继续每天学习新单词50个，增加复习频率，确保词汇量的增长。同时，将已学词汇与上下文相结合，加深印象。
- **听力**：每天至少练习40分钟的听力，包括模拟试题。尝试找到不同的材料来源（如英语电影、TED演讲

## 总结

1. Base模型如果参数量足够多，规模足够大，这样积累的知识较多的情况下更适合一些合成数据的任务，比如基于GPT-3合成Phi模型的预训练数据，因为预训练模型不会在回答上有“偏向”，知识更广泛。
2. Instruct模型更适合做指令遵从任务，也就是大部分的工作，但是如果指令不明确，可能会导致模型回答的时候根据微调的结果有所偏向。
3. Reasoning模型更适合复杂任务，因为会生成很长的推理过程，很费时间，如果任务过于简单，Instruct模型就能完成。